## Data Generation

In [1]:
import os
import pandas as pd

In [3]:
input_file = r'../data/yahoo_top_products_click_popularity_20200201.txt'

In [4]:
df_train = pd.read_csv(input_file, sep=r'\t')

C:\Users\echon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
df_train.head()

,Parent L1 Description,Parent L2 Description,MID,Merchant Name,OID,Title,Description,Manufacturer,Price,Min Offer Price 30 Day,Price Change,Click Popularity
0,Pet Supplies,More Pet Supplies,248942,Chewy.com,9151715234,Wisdom Panel Health Breed & Health Identificat...,Wisdom Panel Health Breed & Health Identificat...,Wisdom Panel,149.99,89.99,66.67,35415.0480
1,Clothing & Accessories,Men's Clothing,313027,eBay PLA US,11114337071,Girl Scout Cookies 2019-20 New Cookies are in!...,Girl Scout Cookies 2019-20 New Cookies are in!...,Girl Scouts,48.00,48.00,0.00,12215.2644
2,Clothing & Accessories,Men's Clothing,134716,MensUSA.com,5651235052,Classic Long Royal Blue Fashion Zoot Suit,"""This Zoot Suit is as nice and unique as it ge...",mensusa,139.00,139.00,0.00,8317.7750
3,'N/A','N/A',76071,Joe's New Balance Outlet,9756405838,New Balance Women's FuelCore NERGIZE Shoes Bla...,Slip on the FuelCore NERGIZE women's training ...,New Balance,38.99,38.99,0.00,6132.8540
4,Clothing & Accessories,Handbags & Luggage,31851,Kohl's,5681651813,"Stone & Co. Irene Leather Hobo, Grey",Watch the product video here. Stone & Co. embo...,Stone & Co.,99.00,69.30,42.85,5251.5413


In [6]:
null_rows = (df_train['Title'].isnull()) \
            | (df_train['Parent L1 Description'].isnull()) \
            | (df_train['Parent L2 Description'].isnull()) \
            | (df_train['Price'].isnull()) \
            | (df_train['Click Popularity'].isnull())
df_train = df_train[~null_rows]



In [17]:
# MAX LENGTH substitution
#np.max(df_train['Title'].apply(lambda x : len(x)).to_list())
MAX_LENGTH = 256
df_train = df_train[~(df_train.Title.apply(lambda x : len(x)) > MAX_LENGTH)]

In [18]:
SAMPLE_FRAC = 0.1
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9487)

In [19]:
# 去除不必要的欄位並重新命名兩標題的欄位名
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['Title', 'Price', 'Click Popularity']]
df_train.head()

,Title,Price,Click Popularity
0,Buy Art For Less Coastal Nautical Sea Life Sea...,48.99,0.1932
1,Safavieh Light Blue/Dark Blue Adirondack 10 ft...,68.50,0.5798
2,Funny Prank Blurry Eye Chart Exam T Shirt,16.99,0.1932
3,"Opinion Ciatti Gaga 35.43"" H x 45.28"" W Desk H...",1369.99,0.7730
4,J. Crew Wool Cardigan Sweater: Black Solid Swe...,21.99,0.3865


In [ ]:
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用
df_train.to_csv("train.tsv", sep="\t", index=False)

print("訓練樣本數：", len(df_train))
df_train.head()